# Consiguiendo los datos con Tweepy

**Tweepy** es una libreria escrita en `Python` para manejar la red social `Twitter` de una forma sencilla y directa. Con [Tweepy](http://www.tweepy.org/) podeis acceder a la `API` de Twitter y realizar busquedas, actualizar vuestro perfil o analizar a vuestros seguidores.

Tweepy es relativamente intuitiva y de fácil uso. Lo primero que necesitamos es instalar la librería en python, que como la mayoría de librerías puedes instalar con `pip` o `easy_install`. Abrid vuestra consola de comandos e instalad Tweepy con alguna de las siguientes órdenes:

**Instalación con pip:**
```bash
pip search tweepy # para buscar si esta en los repositorios
pip install tweepy
```
**Instalación con easy_install:**
```bash  
easy_install tweepy
```
Tened en cuenta que necesitais registrar vuestra aplicación en Twitter para comunicaros con la API, concretamente en la web de [Twitter para desarrolladores](https://apps.twitter.com/). Una vez os habeis registrado con vuestra cuenta y elegido los permisos correspondientes para la app, en `Keys and Access Tokens` podeis encontrar dos claves; API_key y API secret (en adelante `consumer_key` y `consumer_secret`, respectivamente). Necesitareis además generar los tokens; `access_token` y `access_secret`. Estas cuatro claves son necesarias para comunicaros con la red social. 

Con tal de no tener que copiar las claves cada vez que intenteis hacer una busqueda, os recomiendo guardarlas en un archivo vacio llamado `passw.py`. El contenido de este debe ser solo vuestras claves siguiendo este formato:
```python
consumer_key = 'sustituid el contenido de estas comillas por vuestro consumer key'
consumer_secret = 'sustituid el contenido de estas comillas por vuestro consumer secret'
access_token = 'sustituid el contenido de estas comillas por vuestro access token'
access_secret = 'sustituid el contenido de estas comillas por vuestro access secret'
```
Una vez hemos instalado Tweepy y generado el archivo con las contraseñas (el cual guardaremos en la carpeta de trabajo) podemos importarlos a nuestro script para su uso. 

In [28]:
import tweepy
from passw import consumer_key, consumer_secret, access_token, access_secret
import numpy as np
import pandas as pd

Ejecutando el código que se encuentra a continuación nos comunicaremos con Twitter para mandarle nuestras credenciales. Twitter comprobará que hemos creado correctamente una aplicación y obtendremos una autorización para conectarnos con la API. Finalmente obtenemos un objeto llamado `api` sobre el cual podemos ejecutar las diferentes funciones que nos ofrece Tweepy. 

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

Una vez hemos accedido, hay muchas cosas que podemos hacer:
- enviar un tuit
- seguir o dejar de seguir a otros
- actualizar el perfil
- obtener información de un usuario
- obtener todos los tuits de un usuario
- ver todos los seguidores de otro usuario
- hacer búsquedas de tuits
- guardar todos los tuits que van apareciendo en directo
- ...   

A modo de demostración, podemos obtener de manera sencilla la siguiente información:

In [18]:
usuario = api.get_user('juan_qenk')
print('Nombre de Usuario: ', usuario.screen_name)
print('Fecha de creación:', usuario.created_at)
print('Descripción:', usuario.description)
print('Número de seguidores:', usuario.followers_count)
print('Localización:', usuario.location)

Nombre de Usuario:  juan_qenk
Fecha de creación: 2012-04-20 16:06:21
Descripción: Biólogo capaz de hacer la fotosíntesis incluso en días nublados. Habla más de 100 idiomas, todos inventados por él en una sola noche.
Número de seguidores: 921
Localización: Agost


Twitter tiene una normativa que regula la frecuencia máxima de peticiones, así como la cantidad máxima de tweets que se pueden extraer, [rate limiting](https://developer.twitter.com/en/docs/basics/rate-limits). Es importante cumplir estos límites para evitar ser bloqueado.

Para el trabajo de análisis que se quiere realizar, es conveniente recuperar tantos tweets como sea posible, al menos unos 1000 por búsqueda. Para ello podemos usar el objeto `cursor` de Tweepy. Este objeto nos permite paginar los resultados ajustandonos a un número de tuits máximo que podemos elegir. 

In [19]:
query = 'biotechnology'
max_tweets = 1000
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]

Acabamos de generar una lista con todos los tuits que coinciden con la palabra *biotechnology*. Hemos ajustado el número máximo de tuits en 1000, con lo que, si hemos cumplido todos los requisitos que exige la API, 
```python 
len(searched_tweets) == 1000
```


Esta lista de tuits no contiene solo el texto del tuit, es un objeto complejo que tiene mucha información. Si tomamos el primer tuit de la lista, `searched_tweets[0]`, y lo imprimimos en pantalla, podemos ver que toda la información se encuentra en formato json. Json es un formato de texto ligero para el intercambio de datos muy común en internet. 

In [21]:
searched_tweets[0]

Status(_api=<tweepy.api.API object at 0x7f19b4ea2400>, _json={'created_at': 'Tue Dec 26 18:59:27 +0000 2017', 'id': 945730796517777408, 'id_str': '945730796517777408', 'text': "RT @VeeKaMajozi: Hi y'all.\nI have ND in Biotechnology with 2 years experience. Please 🙏 RT for me and help me find a job. I'm willing to re…", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'VeeKaMajozi', 'name': 'IG:vee_majozi', 'id': 1607599200, 'id_str': '1607599200', 'indices': [3, 15]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Lite</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 817700425457664001, 'id_str': '817700425457664001', 'name': 'Lumkile ka xhoba', 'screen_name': 'MosheLumkile', 'location': 'South africa', '

La salida del último comando puede resultar dificilmente legible. Sin embargo podemos leer el contenido de un modo menos doloroso si usamos la función `pprint()`.

In [22]:
import json
import pprint
pprint.pprint(searched_tweets[0]._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Dec 26 18:59:27 +0000 2017',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [],
              'user_mentions': [{'id': 1607599200,
                                 'id_str': '1607599200',
                                 'indices': [3, 15],
                                 'name': 'IG:vee_majozi',
                                 'screen_name': 'VeeKaMajozi'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 945730796517777408,
 'id_str': '945730796517777408',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'retweet_count': 88,
 'retweeted': False,
 'retweeted_status': {'contributors': None,
                      'coordinates': None

Podemos acceder a un gran número de información tanto del tuit como del usuario que lo creó. 


In [27]:
print('Fecha: ', searched_tweets[0].created_at)
print('Usuario: @' + searched_tweets[0].user.screen_name)
print('Nombre: ', searched_tweets[0].user.name)
print('Descripción: ', searched_tweets[0].user.description)
print('Localización: ', searched_tweets[0].user.location)
print('Verificado: ', searched_tweets[0].user.verified)
print('Seguidores: ', searched_tweets[0].user.followers_count)
print('Texto: ', searched_tweets[0].text)
print('RT: ', searched_tweets[0].retweet_count)
print('Favs: ', searched_tweets[0].favorite_count)

Fecha:  2017-12-26 18:59:27
Usuario: @MosheLumkile
Nombre:  Lumkile ka xhoba
Descripción:  Confident and visionary, politically and business minded
Localización:  South africa
Verificado:  False
Seguidores:  283
Texto:  RT @VeeKaMajozi: Hi y'all.
I have ND in Biotechnology with 2 years experience. Please 🙏 RT for me and help me find a job. I'm willing to re…
RT:  88
Favs:  0


In [30]:
columns = ['Usuario','Nombre', 'Fecha', 'Tuit', 'RT', 'Fav', 'Verificado']
df = pd.DataFrame(columns=columns)
for tuit in searched_tweets:
    Fecha = tuit.created_at
    Usuario = tuit.user.screen_name
    Nombre = tuit.user.name
    Verificado= tuit.user.verified
    Texto= tuit.text
    RT= tuit.retweet_count
    Favs= tuit.favorite_count
    df.loc[len(df)]=[Usuario, Nombre, Fecha, Texto, RT, Favs, Verificado ] 
    

In [32]:
df.tail()

,Usuario,Nombre,Fecha,Tuit,RT,Fav,Verificado
995,Mahendr82835960,Mahendra Kumar Raj,2017-12-24 11:44:03,RT @ShubhamSaxenaIN: Karnataka is the fastest ...,13,0,False
996,Biotherapeutic1,Biotherapeutics,2017-12-24 11:12:09,#biotherapeutics jobs lifescience CIVIL ENGINE...,0,0,False
997,Biotherapeutic1,Biotherapeutics,2017-12-24 11:12:08,#biotherapeutics jobs lifescience Sr SAP PPDS ...,0,0,False
998,HelloMylesHere,Myles Wirth,2017-12-24 11:11:21,"And so each time, Mario sets off through the r...",0,5,False
999,NandkishorBhan2,Nandkishor Bhandekar,2017-12-24 11:02:49,RT @rashtrapatibhvn: Cross-fertilisation and d...,95,0,False


In [33]:
df.to_pickle('biotechnology.pkl')

Siguiendo el sistema anterior se realizaron las siguientes busquedas:
   
- vacunas ("Genetically modified" OR GMO OR transgenic) AND (vaccine OR vaccination) lang:en exclude:retweets
- medicinas ("Genetically modified" OR GMO OR transgenic) AND (medicina OR drug) lang:en exclude:retweets
- alimentos funcionales ("Genetically modified" OR GMO OR transgenic) AND ("functional food") lang:en exclude:retweets
- arroz dorado ("Genetically modified" OR GMO OR transgenic) AND ("golden rice") lang:en exclude:retweets
- cultivos ("Genetically modified" OR GMO OR transgenic) AND (crops) lang:en exclude:retweets
- pesticidas ("Genetically modified" OR GMO OR transgenic) AND (pesticides) lang:en exclude:retweets
- medio ambiente ("Genetically modified" OR GMO OR transgenic) AND (environment) lang:en exclude:retweets
- bioremediación ("Genetically modified" OR GMO OR transgenic) AND (bioremediation) lang:en exclude:retweets
- cancer ("Genetically modified" OR GMO OR transgenic) AND (cancer) lang:en exclude:retweets
- alergias ("Genetically modified" OR GMO OR transgenic) AND (allergy OR allergies) lang:en exclude:retweets
- Tranferencia de genes ("Genetically modified" OR GMO OR transgenic) AND ("gene transfer") lang:en exclude:retweets
- Desigualdad ("Genetically modified" OR GMO OR transgenic) AND (inequality) lang:en exclude:retweets
- Sociedad ("Genetically modified" OR GMO OR transgenic) AND (society) lang:en exclude:retweets
- Biodiversidad ("Genetically modified" OR GMO OR transgenic) AND (biodiversity) lang:en exclude:retweets
   
Además, usando los datos en directo, se han filtrado los siguientes tuits:   

    ("Genetically modified" OR GMO OR transgenic) lang:en exclude:retweets